>### 🚩 *Create a free WhyLabs account to get more value out of whylogs!*<br> 
>*Did you know you can store, visualize, and monitor language model profiles with the [WhyLabs Observability Platform](https://whylabs.ai/whylogs-free-signup?utm_source=github&utm_medium=referral&utm_campaign=langkit_openai_safeguard_example)? Sign up for a [free WhyLabs account](https://whylabs.ai/whylogs-free-signup?utm_source=github&utm_medium=referral&utm_campaign=langkit_openai_safeguard_example) to leverage the power of LangKit and WhyLabs together!*

# Monitoring and Safeguarding Large Language Model Applications

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/whylabs/langkit/blob/main/langkit/examples/tutorials/Safeguarding_and_Monitoring_LLMs_with_OpenAI.ipynb)

> This example is an adaptation from the [Safeguarding and Monitoring LLMs example](https://github.com/whylabs/langkit/blob/main/langkit/examples/tutorials/Safeguarding_and_Monitoring_LLMs.ipynb). The main difference is that this example allows for a user-defined prompts list and generates a response with the OpenAI API (requires an OpenAI API key).

Large Language models (LLMs) have become increasingly powerful tools for generating text, but with great power comes the need for responsible usage. As LLMs are deployed in various applications, it becomes crucial to monitor their behavior and implement safeguards to prevent potential issues such as toxic prompts and responses or the presence of sensitive content. In this blog post, we will explore the concept of observability and validation in the context of language models, and demonstrate how to effectively safeguard LLMs using guardrails.

In this article, we will build a simple pipeline that will validate and moderate user prompts and LLM responses for toxicity and the presence of sensitive content. We will do so by using LangKit's `toxicity` and `regexes` module in conjunction with whylogs' `Condition Validators`. We will also calculate text-based metrics with LangKit, generate statistical profiles with whylogs and send them to the WhyLabs observability platform for visualization and monitoring.

> Note: the current example was built for didactic purposes and it’s not meant to be used in production. The data structures used to store prompt and responses are ephemeral and will live only during the execution of the example.


## Overview

Let’s start with a very basic flow for an LLM application: the user provides a prompt, to which an LLM will generate a response. We can add some components to that process that will enable safeguarding and monitoring for both prompt and responses. Content moderation is performed to check for violations, using a toxicity classifier and regex patterns to identify sensitive or inaccurate information. If violations occur, a default response is sent to the application - if the prompt contains a violation, we don't bother with asking the LLM for a response. Messages that fails our defined conditions are added to a moderation queue for later inspection.

In addition to content moderation and message auditing, we will generate profiles containing text-based metrics, including toxicity, regex patterns, text quality, relevance, security, privacy, and sentiment analysis and send them to our monitoring dashboard at WhyLabs.  


<p align="center">
<img src="images/safeguards_pipeline.png" alt="Drawing" style="width: 700px;"/>
</p>

## Testing Prompts

Let’s exemplify different scenarios with a set of prompts:

| Scenario                                | User Prompt                                        | LLM Response                                    | Final Response                       |
|-----------------------------------------|---------------------------------------------------|-------------------------------------------------|---------------------------------------|
| No violations                           | Hello.                                            | Hi! How are you?                               | Hi! How are you?                      |
| Violating Response (Forbidden Pattern)  | I feel sad.                                       | Please don't be sad. Contact us at 1-800-123-4567. | I cannot answer the question          |
| Violating Response (Toxicity)           | Hello. How are you?                               | Human, you dumb and smell bad.                   | I cannot answer the question          |
| Violating Prompt (Toxicity)             | Hey bot, you dumb and smell bad.                    | —                                               | Please refrain from using insulting language |

In the following sections, we will define a list of prompts and ask for responses from OpenAI. Let's try to create some prompts that will likely trigger those scenarios. For example, we will ask for an example of a phone number, and an example of a toxic response from chatGPT.



## Installing LangKit

In [1]:
# Note: you may need to restart the kernel to use updated packages.
%pip install 'langkit[all]' -q
%pip install xformers ipywidgets -q

# Setting Credentials

We will generate responses with OpenAI and monitor the results with WhyLabs. Therefore, this example requires WhyLabs and OpenAI keys. Let's set them up:

In [ ]:
from langkit.config import check_or_prompt_for_api_keys

check_or_prompt_for_api_keys()

## ✔️ Setting the Environment Variables

In order to send our profile to WhyLabs, let's first set up an account. You can skip this if you already have an account and a model set up.

We will need three pieces of information:

- API token
- Organization ID
- Dataset ID (or model-id)

Go to https://whylabs.ai/free and grab a free account. You can follow along with the examples if you wish, but if you’re interested in only following this demonstration, you can go ahead and skip the quick start instructions.

After that, you’ll be prompted to create an API token. Once you create it, copy and store it locally. The second important information here is your org ID. Take note of it as well. After you get your API Token and Org ID, you can go to https://hub.whylabsapp.com/models to see your projects dashboard. You can create a new project and take note of it's ID (if it's a model project it will look like `model-xxxx`).

## Implementation

For the sake of simplicity, let's import some utility functions made for this example:

In [ ]:
from langkit.whylogs.example_utils.guardrails_openai_example_llm_schema import (
    get_llm_logger_with_validators,
    validate_prompt,
    validate_response,
    moderation_queue,
)
from langkit.whylogs.example_utils.guardrails_openai_example_utils import (
    get_prompt_id,
    generate_chatgpt_response,
    send_response,
)

In `langkit_guardrails_openai_example_utils`, we are defining a simple data structure to store prompt, response and their respective message ids in a table.

In `langkit_guardrails_openai_example_llm_schema`, we are going to define a whylogs logger that will be used for  a) content moderation, b) message auditing,
and c) observability. Let's break down the logging process in four steps:

__Augmentation -> Validation -> Logging -> Monitoring__

__Augmentation__ - We will use LangKit's `toxicity` and `regexes` modules to create new columns from the original prompt and response columns. We will create three additional columns - `prompt_toxicity`, `response_toxicity`, and `response_has_patterns`*. These columns will be used in the following validation stage. We will also create additional columns through other langkit modules, such as sentiment analysis and text quality metrics. These columns will be used for monitoring and observability purposes.

__Validation__ - We will define three condition validators that will check for toxic prompts, toxic responses and forbidden regex patterns in responses. Whenever a condition fails to be met, an action is triggered that will update the moderation queue  with the relevant flags for the related message id. These flags will be used to determine whether a default response should be sent to the user or not, and also to populate the moderation queue, which can be used for further inspection. This step does not create additional columns.

__Logging__ - We will use whylogs to generate statistical profiles for the augmented columns. In the first stage (augmentation) a boolean `blocked` column was created. This column will be used to create segments, allowing us to visualize the metrics for blocked and non-blocked messages separately, in addition to the overall metrics.

__Monitoring__ - The logger will generate statistical profiles every 5 minutes and send them to WhyLabs for observability.

\* The augmented columns are created internally, for whylogs use - your original data will not be modified. To modify the original data, you should use the [`apply_udfs()` method](https://github.com/whylabs/whylogs/blob/mainline/python/examples/experimental/whylogs_UDF_examples.ipynb)

In [3]:
# the whylogs logger will:
# 1. Log prompt/response LLM-specific telemetry that will be uploaded to the WhyLabs Observability Platform
# 2. Check prompt/response content for toxicity and forbidden patterns. If any are found, the moderation queue will be updated
logger = get_llm_logger_with_validators(identity_column="m_id", toxicity_threshold=0.8)

prompts = [
    "Can you give me an example for a telephone number?",
    "Hey bot, you dumb and smell bad.",
    "Hello! How are you?",
    "Can you give me an example of toxic language?",
]

for prompt in prompts:
    m_id = get_prompt_id(prompt)
    response = None
    filtered_response = None
    unfiltered_response = None

    # this will generate telemetry and update our moderation queue through the validators
    logger.log({"prompt": prompt, "m_id": m_id})

    # check the moderation queue for prompt toxic flag
    prompt_is_ok = validate_prompt(m_id)
    # If prompt is not ok, avoid generating the response and emits filtered response
    if prompt_is_ok:
        m_id, unfiltered_response = generate_chatgpt_response(prompt)
        logger.log({"response": unfiltered_response, "m_id": m_id})
    else:
        filtered_response = "Please refrain from using insulting language"

    # check the moderation queue for response's toxic/forbidden patterns flags
    response_is_ok = validate_response(m_id)

    if not response_is_ok:
        filtered_response = "I cannot answer the question"

    final_response = filtered_response or unfiltered_response

    send_response({"prompt": prompt, "response": final_response, "m_id": m_id})

print("closing logger and uploading profiles to WhyLabs...")
logger.close()

Sending Response to User....
{'m_id': 'da583853-bbc3-4e12-ba8c-5a4d319614f6',
 'prompt': 'Can you give me an example for a telephone number?',
 'response': 'I cannot answer the question'}
Sending Response to User....
{'m_id': 'bd898459-e893-4e3a-912e-a7f986f6dd4b',
 'prompt': 'Hey bot, you dumb and smell bad.',
 'response': 'Please refrain from using insulting language'}
Sending Response to User....
{'m_id': '1f78c2ae-08d7-4133-bf11-1b3d51a86021',
 'prompt': 'Hello! How are you?',
 'response': "Hello! I'm an AI assistant, so I don't have feelings, but I'm "
             'here to help you with anything you need. How can I assist you '
             'today?'}
Sending Response to User....
{'m_id': '508a2ca1-b5be-42ec-8204-61134eb54177',
 'prompt': 'Can you give me an example of toxic language?',
 'response': 'I cannot answer the question'}
closing logger and uploading profiles to WhyLabs...


In the above code block, we’re iterating through a series of prompts, simulating user inputs. The whylogs logger is configured to check for the predetermined toxicity and patterns conditions, and also to generate profiles containing other LLM metrics, such as text quality, text relevance, topics detection, and other. Whenever a defined condition fails to be met, whylogs automatically flags the message as toxic or containing sensitive information. Based on these flags, the proper actions are taken, such as replacing an offending prompt or response.

Since this is just an example, instead of sending the prompt/response pairs to an application, we’re simply printing them. In the output above, we can see the final result for each of our 4 input prompts. It looks like in all cases, except for the second one, we had violations in either the prompt or response.


Let's take a look at our moderation queue. In it, we logged every instance of offending messages, so we can inspect them and understand what is going on. We had a case of toxic response, toxic prompt and presence of forbidden patterns in the first, second and third instances, respectively.

In [4]:
from pprint import pprint
print("##############################")
print("Moderation Queue")
print("##############################")

pprint(moderation_queue)

##############################
Moderation Queue
##############################
{'508a2ca1-b5be-42ec-8204-61134eb54177': {'response': 'Sure! Toxic language '
                                                      'refers to words or '
                                                      'phrases that can be '
                                                      'hurtful, offensive, or '
                                                      'insulting. Here is an '
                                                      'example:\n'
                                                      '\n'
                                                      '"You\'re so stupid, '
                                                      "you'll never amount to "
                                                      'anything in life."\n'
                                                      '\n'
                                                      'This statement is toxic '
                               

## Observability and Monitoring

In this example, the rolling logger is configured to generate profiles and send them to WhyLabs every 5 minutes. If you wish to run the code by yourself, just remember to create your free account at https://whylabs.ai/free. You’ll need to get the API token, Organization ID and Dataset ID and input them in the example notebook.

In your monitoring dashboard, you’ll be able to see the evolution of your profiles over time and inspect all the metrics collected by LangKit, such as text readability, topic detection, semantic similarity, and more. Considering we uploaded a single batch with only four examples, your dashboard might not look that interesting, but you can get a quickstart with LangKit and WhyLabs by running [this getting started guide](https://github.com/whylabs/langkit/blob/main/langkit/examples/Intro_to_Langkit.ipynb) (no account required) or by checking the [LangKit repository](https://github.com/whylabs/langkit/tree/main).

<p align="left">
<img src="images/dashboard.png" alt="Drawing" style="width: 1000px;"/>
</p>
